In [1]:
%cd ..

/home/alta/Conversational/OET/al826/2022/shortcuts/data_pruning


In [1]:
from datasets import load_dataset

dataset = load_dataset("hans")


Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset hans downloaded and prepared to /home/alta/Conversational/OET/al826/.cache/huggingface/datasets/hans/plain_text/1.0.0/452e93cf5383f5ae39088254215b517d0da98ccaaf0af8f7ab04d8f23f67dbd9. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import time

for i in dataset['train']:
    print(i)
    time.sleep(1)

{'premise': 'The doctors supported the scientist .', 'hypothesis': 'The scientist supported the doctors .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NNS doctors)) (VP (VBD supported) (NP (DT the) (NN scientist))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN scientist)) (VP (VBD supported) (NP (DT the) (NNS doctors))) (. .)))', 'binary_parse_premise': '( ( The doctors ) ( ( supported ( the scientist ) ) . ) )', 'binary_parse_hypothesis': '( ( The scientist ) ( ( supported ( the doctors ) ) . ) )', 'heuristic': 'lexical_overlap', 'subcase': 'ln_subject/object_swap', 'template': 'temp1'}
{'premise': 'The athletes introduced the tourist .', 'hypothesis': 'The tourist introduced the athletes .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NNS athletes)) (VP (VBD introduced) (NP (DT the) (NN tourist))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN tourist)) (VP (VBD introduced) (NP (DT the) (NNS athletes))) (. .)))', 'binary_parse_premise': '( ( Th

{'premise': 'The actor avoided the authors .', 'hypothesis': 'The authors avoided the actor .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN actor)) (VP (VBD avoided) (NP (DT the) (NNS authors))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NNS authors)) (VP (VBD avoided) (NP (DT the) (NN actor))) (. .)))', 'binary_parse_premise': '( ( The actor ) ( ( avoided ( the authors ) ) . ) )', 'binary_parse_hypothesis': '( ( The authors ) ( ( avoided ( the actor ) ) . ) )', 'heuristic': 'lexical_overlap', 'subcase': 'ln_subject/object_swap', 'template': 'temp1'}
{'premise': 'The scientist admired the actors .', 'hypothesis': 'The actors admired the scientist .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN scientist)) (VP (VBD admired) (NP (DT the) (NNS actors))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NNS actors)) (VP (VBD admired) (NP (DT the) (NN scientist))) (. .)))', 'binary_parse_premise': '( ( The scientist ) ( ( admired ( the actors ) ) . ) )

{'premise': 'The doctors introduced the tourists .', 'hypothesis': 'The tourists introduced the doctors .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NNS doctors)) (VP (VBD introduced) (NP (DT the) (NNS tourists))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NNS tourists)) (VP (VBD introduced) (NP (DT the) (NNS doctors))) (. .)))', 'binary_parse_premise': '( ( The doctors ) ( ( introduced ( the tourists ) ) . ) )', 'binary_parse_hypothesis': '( ( The tourists ) ( ( introduced ( the doctors ) ) . ) )', 'heuristic': 'lexical_overlap', 'subcase': 'ln_subject/object_swap', 'template': 'temp1'}
{'premise': 'The manager admired the senator .', 'hypothesis': 'The senator admired the manager .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN manager)) (VP (VBD admired) (NP (DT the) (NN senator))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN senator)) (VP (VBD admired) (NP (DT the) (NN manager))) (. .)))', 'binary_parse_premise': '( ( The manager ) ( ( 

KeyboardInterrupt: 

In [12]:
from src.system_loader import EnsembleLoader, SystemLoader
from src.utils.evaluation import get_seed_accuracy, get_accuracy

system = EnsembleLoader('trained_models/hpc/baseline') 
seed_preds  = system.load_seed_preds('paws', mode='test')
labels = SystemLoader.load_labels('paws', mode='test')
print(get_seed_accuracy(seed_preds, labels))

Reusing dataset paws (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338)


  0%|          | 0/3 [00:00<?, ?it/s]

[91.3375, 91.57499999999999, 91.4875, 91.9625, 91.2]
(91.51249999999999, 0.2589642832515712)


In [13]:
import numpy as np

sample_ids = [k for k in labels]
seed_1 = np.array([seed_preds[0][k] for k in sample_ids])
seed_2 = np.array([seed_preds[1][k] for k in sample_ids])
seed_3 = np.array([seed_preds[2][k] for k in sample_ids])
seed_4 = np.array([seed_preds[3][k] for k in sample_ids])
seed_5 = np.array([seed_preds[4][k] for k in sample_ids])
labels_np = np.array([labels[k]     for k in sample_ids])

In [14]:
print(len(labels_np))

8000


In [15]:
incorrect_1 = [k for k in sample_ids if seed_preds[0][k] != labels[k]]
incorrect_2 = [k for k in sample_ids if seed_preds[1][k] != labels[k]]

In [16]:
overlap = len(set(incorrect_1) & set(incorrect_2))
union   = len(set(incorrect_1) | set(incorrect_2))

print(overlap/union)

0.5463800904977375


In [17]:
overlap = (seed_4 == labels_np) == (seed_1 == labels_np)
print(overlap.sum()/labels_np.shape[0])

0.95075


In [ ]:
from src.